In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Function to load and preprocess images
def load_images(folder_path):
    images = []
    labels = []

    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            # Flatten the image to a 1D array
            images.append(img.flatten())

            # Assign labels based on classes
            if "non-defected" in filename:
                labels.append(0)  # Class 1: non-defected
            elif "breakage" in filename:
                labels.append(1)  # Class 2: defected (breakage)
            elif "bubbles" in filename:
                labels.append(2)  # Class 3: defected (presence of bubbles)
            elif "discolorization" in filename:
                labels.append(3)  # Class 4: defected (discolorization)

    return np.array(images), np.array(labels)

In [ ]:
# Load and preprocess images
dataset_folder = 'ImgsData'
images, labels = load_images(dataset_folder)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
#CNN

# Define hyperparameters
input_shape = (64, 64, 1)  # Adjust based on your image size and channels
num_classes = 4
learning_rate = 0.001
batch_size = 32
epochs = 50

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(
    optimizer=Adam(learning_rate),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Example: Use EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model (assuming you have training data X_train, y_train and validation data X_val, y_val)
history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:

y_pred = np.argmax(model.predict(X_test), axis=1)

# Calculate overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Overall Accuracy: {accuracy:.4f}')

# Calculate precision, recall, and F1 score for each class
class_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3']
classification_rep = classification_report(y_test, y_pred, target_names=class_names)
print('Classification Report:\n', classification_rep)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', conf_matrix)


In [ ]:
# Function to perform defect detection on a new image
def detect_defect(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    flattened_img = img.flatten().reshape(1, -1)
    predictions = model.predict(flattened_img)

    class_labels = {
        0: "Non-defected",
        1: "Defected with Breakage",
        2: "Defected with Presence of Bubbles",
        3: "Defected with Discolorization"
    }

    predicted_class = predictions[0]  # Assuming predictions is a list with class probabilities

    print(f"Predicted Class: {class_labels[predicted_class]}")
